In [4]:
import hydrobr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [6]:
def cities(cities, state):
  if len(cities) > 0:
    table = []
    for i in range(0, len(cities)):
      station = hydrobr.get_data.ANA.list_prec_stations(state = state, city=cities[i])
      table.append(station)
    final_table = pd.concat(table)
    display(final_table)
    return final_table
  else:
    station = hydrobr.get_data.ANA.list_prec_stations(state = state)
    display(station)
    return station

In [7]:
def data_gantt(table, code = 0):
  if code == 0:
    stations = table.Code.to_list()
    data = hydrobr.get_data.ANA.prec_data(stations)
    id = table["Code"]
    name = table["Name"]
    responsible = table["Responsible"]
    id_list = []
    name_list = []
    responsible_list = []
    for i in range(0, len(table)):
      id_list.append(id.iloc[i])
    for i in range(0, len(table)):
      name_list.append(name.iloc[i])
    for i in range(0, len(table)):
      responsible_list.append(responsible.iloc[i])
    for i in range(0, len(id_list)):
      data = data.rename(columns={id_list[i]: name_list[i] + " " + "(" + responsible_list[i] + ")"})
    return data
  if code == 1:
      stations = table.Code.to_list()
      data = hydrobr.get_data.ANA.prec_data(stations)
      return data

In [ ]:
city_list = ['MUCURI', 'RODELAS'] # Choose the cities of your nterest. Leave it empty to get all cities from the state below.
state = 'BAHIA'                   # Choose the state regarding the cities above. You can select just one per time.
table = cities(city_list, state)

In [25]:
station = gpd.GeoDataFrame(table, geometry = gpd.points_from_xy(table['Longitude'], table['Latitude']))

In [21]:
raster = rasterio.open('/content/gdrive/MyDrive/TIFS/2021-02-09.tif') #Replace it with your image path and name

In [ ]:
data = data_gantt(table, 1) # 0 will display station names on the Gantt chart, and 1 will display their codes.

In [ ]:
img, axis = plt.subplots(figsize = (10,8))

show(raster, ax = axis)
station.plot(ax = axis, color = 'red', alpha = 0.4)

In [ ]:
gantt = hydrobr.Plot.gantt(data)

gantt.update_layout(
    autosize=True,
    width=1300,
    height=500,
    xaxis_title = 'Year',
    yaxis_title = 'Stations',
    font=dict(family="Courier New", size=15))

display(gantt)

In [ ]:
#Each station occurrence for each responsible
station = hydrobr.get_data.ANA.list_prec_stations(state ='BAHIA')
station["Responsible"].value_counts()